In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import operator

d1 = pd.read_csv('user_mat_rating_modified_all.csv')

material_unique=d1['Material'].unique().tolist()
wholesaler_unique=d1['Wholesaler'].unique().tolist()

mudi={}
c=0
for i in material_unique:
    mudi[i]=c
    c=c+1

wudi={}
c=0
for i in wholesaler_unique:
    wudi[i]=c
    c=c+1

for i in range(len(d1['Material'])):
    d1['Material'][i]=mudi[d1['Material'][i]]

for i in range(len(d1['Material'])):
    d1['Wholesaler'][i]=wudi[d1['Wholesaler'][i]]

d1['rating']=d1['rating']+1
d1['logarithm_HLs'] = np.log10(d1['rating'])
max_HL=max(d1['logarithm_HLs'])
d1['logarithm_HLs']=d1['logarithm_HLs']/max_HL

x=d1[['Wholesaler','Material']]
y=d1[['logarithm_HLs']]

num_users=len(d1['Wholesaler'].unique())
num_materials=len(d1['Material'].unique())

<ipython-input-2-d14229e16551>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1['Material'][i]=mudi[d1['Material'][i]]
<ipython-input-2-d14229e16551>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1['Wholesaler'][i]=wudi[d1['Wholesaler'][i]]


In [3]:
d1

,Wholesaler,Material,Trend,gaps,freq,rating,logarithm_HLs
0,0,0,"[1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.472222,41.111111,28.924528,0.420128
1,0,1,"[1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...",2.560000,28.888889,12.284722,0.313203
2,0,2,"[1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1]",0.571429,66.666667,117.666667,0.595333
3,0,3,"[1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...",2.869565,25.842697,10.005788,0.287582
4,0,4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ...",6.250000,14.772727,3.363636,0.151463
...,...,...,...,...,...,...,...
8323,178,23,"[1, 0, 0, 1, 0]",1.500000,40.000000,27.666667,0.414577
8324,178,165,"[1, 0, 0, 0]",3.000000,25.000000,9.333333,0.278895
8325,178,24,"[1, 0, 0, 0]",3.000000,25.000000,9.333333,0.278895
8326,178,150,"[1, 0, 0, 0]",3.000000,25.000000,9.333333,0.278895


In [7]:

EMBEDDING_SIZE = 50

class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_materials, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_materials = num_materials
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.material_embedding = layers.Embedding(
            num_materials,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.material_bias = layers.Embedding(num_materials, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        material_vector = self.material_embedding(inputs[:, 1])
        material_bias = self.material_bias(inputs[:, 1])
        dot_user_material = tf.tensordot(user_vector, material_vector, 2)
        # Add all the components (including bias)
        x = dot_user_material + user_bias + material_bias
        # The sigmoid activation forces the rating to between 0 and 1
        return tf.nn.sigmoid(x)


model = RecommenderNet(num_users, num_materials, EMBEDDING_SIZE)



model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

x=np.array(x)
y=np.array(y)

train_indices = int(0.9 * d1.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

y_train=y_train.squeeze()

history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=64,
    epochs=40,
    validation_dat,
    verbose=1,
)


SyntaxError: positional argument follows keyword argument (<ipython-input-7-1f64c50b841d>, line 62)

In [4]:
# model.save_weights("material_rec", save_format="tf")

In [28]:
# M = RecommenderNet(num_users, num_movies, 50)
# M.load_weights("material_rec")

In [5]:
# f = open('wudi.json',)
# wudi = json.load(f)
# wudi = {int(k):v for k, v in wudi.items()}


# f = open('mudi.json',)
# mudi = json.load(f)
# mudi = {int(k):v for k, v in mudi.items()}


In [5]:



one_user_id=40001857
one_user_wholesaler=[]
for i in range(num_materials):
    one_user_wholesaler.append(one_user_id)
one_user_material=[]
for i in range(num_materials):
    one_user_material.append(i)

one_user=pd.DataFrame({'wholesaler':one_user_wholesaler})
one_user['material']=one_user_material
one_user["wholesaler"] = [wudi[x] for x in one_user["wholesaler"]]

key_list_1 = list(mudi.keys())
val_list_1 = list(mudi.values())

preds={}
for i in range(len(one_user)):
    preds[key_list_1[val_list_1.index(one_user['material'][i])]] = float(model.predict(np.asarray([one_user.iloc[i]]))[0])

preds
#preds=dict(sorted(preds.items(), key=operator.itemgetter(1), reverse=True)[:10])
#ratings_for_all = pd.DataFrame(preds)
#ratings_for_all



{10946: 0.8524261713027954,
 19898: 0.8051149249076843,
 80176: 0.721946120262146,
 66989: 0.7663595676422119,
 3337: 0.6312024593353271,
 48859: 0.5562005043029785,
 29777: 0.5405007004737854,
 33335: 0.6877866387367249,
 5983: 0.7269704341888428,
 3372: 0.8492130637168884,
 11574: 0.8746252059936523,
 20614: 0.7687597274780273,
 59874: 0.6751229763031006,
 71317: 0.7725486755371094,
 63311: 0.6050637364387512,
 9974: 0.8269140720367432,
 48537: 0.7240039706230164,
 37961: 0.5260481238365173,
 59847: 0.6669588088989258,
 3338: 0.46287858486175537,
 10947: 0.760033130645752,
 6013: 0.7888355255126953,
 59848: 0.7984973192214966,
 77313: 0.6918048858642578,
 76623: 0.6095366477966309,
 10965: 0.808964729309082,
 66839: 0.5603294372558594,
 10957: 0.594511091709137,
 59873: 0.5533202290534973,
 3410: 0.834998607635498,
 55357: 0.6517787575721741,
 16550: 0.7053505182266235,
 16519: 0.6260406970977783,
 10611: 0.6092721223831177,
 63627: 0.684059202671051,
 6108: 0.5221279263496399,
 1702

In [32]:
num_movies

179

In [8]:

one_user_id=29606863
one_user_wholesaler=[]
for i in range(num_materials):
    one_user_wholesaler.append(one_user_id)
one_user_material=[]
for i in range(num_materials):
    one_user_material.append(i)

one_user=pd.DataFrame({'wholesaler':one_user_wholesaler})
one_user['material']=one_user_material
one_user["wholesaler"] = [wudi[x] for x in one_user["wholesaler"]]

key_list_1 = list(mudi.keys())
val_list_1 = list(mudi.values())

preds=[]
for i in range(len(one_user)):
    preds.append((float(model.predict(np.asarray([one_user.iloc[i]]))[0]), key_list_1[val_list_1.index(one_user['material'][i])])) 

preds
#preds=dict(sorted(preds.items(), key=operator.itemgetter(1), reverse=True)[:10])
#ratings_for_all = pd.DataFrame(preds)
#ratings_for_all

[(0.6180763244628906, 10946),
 (0.5355531573295593, 19898),
 (0.43868133425712585, 80176),
 (0.48686984181404114, 66989),
 (0.3239476680755615, 3337),
 (0.2612183094024658, 48859),
 (0.2564214766025543, 29777),
 (0.3839702010154724, 33335),
 (0.42914289236068726, 5983),
 (0.6281582117080688, 3372),
 (0.6697764992713928, 11574),
 (0.4924599230289459, 20614),
 (0.36655575037002563, 59874),
 (0.5206385850906372, 71317),
 (0.30612969398498535, 63311),
 (0.5627789497375488, 9974),
 (0.42863330245018005, 48537),
 (0.2358304262161255, 37961),
 (0.37914523482322693, 59847),
 (0.21789467334747314, 3338),
 (0.455477237701416, 10947),
 (0.519219160079956, 6013),
 (0.5437996983528137, 59848),
 (0.37005236744880676, 77313),
 (0.2929005026817322, 76623),
 (0.5399235486984253, 10965),
 (0.2674480676651001, 66839),
 (0.3065860867500305, 10957),
 (0.2749900817871094, 59873),
 (0.5976547598838806, 3410),
 (0.33551448583602905, 55357),
 (0.40314990282058716, 16550),
 (0.2917981743812561, 16519),
 (0.3088

In [9]:
preds.sort(reverse=True)
preds

[(0.6697764992713928, 11574),
 (0.6281582117080688, 3372),
 (0.6180763244628906, 10946),
 (0.5976547598838806, 3410),
 (0.5627789497375488, 9974),
 (0.5437996983528137, 59848),
 (0.5399235486984253, 10965),
 (0.5355531573295593, 19898),
 (0.5206385850906372, 71317),
 (0.519219160079956, 6013),
 (0.5106915235519409, 65660),
 (0.4998422861099243, 74342),
 (0.49326398968696594, 11766),
 (0.4924599230289459, 20614),
 (0.48686984181404114, 66989),
 (0.4845283031463623, 73181),
 (0.4758791923522949, 61629),
 (0.47450724244117737, 80417),
 (0.46473169326782227, 66719),
 (0.46292537450790405, 67483),
 (0.4558185338973999, 81408),
 (0.455477237701416, 10947),
 (0.4512636065483093, 80125),
 (0.4503515660762787, 15710),
 (0.4432368874549866, 3530),
 (0.4431793987751007, 66738),
 (0.43868133425712585, 80176),
 (0.437200129032135, 82315),
 (0.43660488724708557, 81410),
 (0.42954280972480774, 70601),
 (0.42914289236068726, 5983),
 (0.42863330245018005, 48537),
 (0.4271184504032135, 63480),
 (0.42065

In [7]:
preds.sort(reverse=True)
preds

[(0.8746252059936523, 11574),
 (0.8524261713027954, 10946),
 (0.8492130637168884, 3372),
 (0.834998607635498, 3410),
 (0.8269140720367432, 9974),
 (0.808964729309082, 10965),
 (0.8051149249076843, 19898),
 (0.7984973192214966, 59848),
 (0.7888355255126953, 6013),
 (0.7849991321563721, 80417),
 (0.7799388766288757, 74342),
 (0.7798489332199097, 61629),
 (0.7763711214065552, 65660),
 (0.7742412090301514, 82315),
 (0.7725486755371094, 71317),
 (0.7687597274780273, 20614),
 (0.7663595676422119, 66989),
 (0.7641681432723999, 73181),
 (0.7638466358184814, 11766),
 (0.760033130645752, 10947),
 (0.754869818687439, 66719),
 (0.7474097013473511, 70601),
 (0.7348345518112183, 81408),
 (0.7346615195274353, 66738),
 (0.7318578958511353, 3530),
 (0.7276047468185425, 70605),
 (0.7269704341888428, 5983),
 (0.72597736120224, 63480),
 (0.7240204811096191, 15710),
 (0.7240039706230164, 48537),
 (0.7234941720962524, 67483),
 (0.721946120262146, 80176),
 (0.7214366793632507, 80125),
 (0.7186084985733032, 7